In [43]:
import os
from os.path import join as opj
import numpy as np

import open3d as o3d
from open3d.web_visualizer import draw

from utils import *
from dataset import *

In [44]:
# set_random_seed(0)

data = dict(
    data_root='data_root',
    partial=False,
    category=['grasp', 'contain', 'lift', 'openable', 'layable', 'sittable',
              'support', 'wrap_grasp', 'pourable', 'move', 'displaY', 'pushable', 'pull',
              'listen', 'wear', 'press', 'cut', 'stab']
)

data_root = data['data_root']
partial = data['partial']
aff_dataset = AffordNetDataset(data_root, 'train', partial=partial)
# aff_dataset = AffordNetDataset(data_root, 'val', partial=partial)
print("Loaded dataset with {} samples".format(len(aff_dataset)))

# import pickle as pkl
# with open(opj('data_root', 'full_shape_%s_data.pkl' % 'train'), 'rb') as f:
#     temp_data = pkl.load(f)

Loaded dataset with 2285 samples


In [45]:
classes_of_interest = ['Knife', 'Bag', 'Bottle', 'Scissors', 'Mug', 'Earphone', 'Hat']

# class -> affordance remapping beacuse we want it agent-centric
aff_remaps = {
    "Knife": {
        "cut": ["handover"],
        "stab": ["handover"],
        "grasp": ["cut", "stab"],
        },
    "Bag": {
        "lift": ["lift"],
        "grasp": ["handover"],
        },
    "Bottle": {
        "openable": ["handover", "wrap"],
        "pourable": ["handover", "wrap"],
        "wrap_grasp": ["pour"],
        },
    "Scissors": {
        "cut": ["handover"],
        "stab": ["handover"],
        "grasp": ["cut", "stab"],
        },
    "Mug": {
        "wrap_grasp": ["handover"],
        "grasp": ["wrap", "pour"],
        },
    "Earphone": {
        "listen": ["handover"],
        "grasp": ["lift"],
        },
    "Hat": {
        "wear": ["handover"],
        "grasp": ["wear"],
        },
}
# Remember to update this:
affordance_classes = ['handover', 'cut', 'stab', 'lift', 'wrap', 'pour', 'wear']

In [46]:
# Initialize new dataset
new_aff_dataset = {
    'semantic_classes': classes_of_interest,
    'affordance_classes': affordance_classes,
    'num_objects': 0,
    'data': dict()
}
# initialize sem_class keys
for sem_class in classes_of_interest:
    new_aff_dataset['data'][sem_class] = dict()


# Loop through the whole dataset and remap affordances of interest
for i, datapoint in enumerate(aff_dataset.all_data):
    if datapoint['semantic class'] not in classes_of_interest:
        continue

    sem_class = datapoint['semantic class']
    shape_id = datapoint['shape_id']
    # print("Shape id ", shape_id, "\nSemantic class: ", sem_class)

    new_datapoint = {
                'coordinates': np.array([], dtype=np.float32),
                'labels': dict()
                }

    # Add pcl coordinates to new datapoint
    new_datapoint['coordinates'] = datapoint['data_info']['coordinate']
    # Get labels of interest for this sem class
    labels_of_interest = aff_remaps[sem_class].keys()
    # print("Labels of interest: ", labels_of_interest)
    non_zero_score = False
    for key in labels_of_interest:
        labels = datapoint['data_info']['label'][key]
        score = labels.sum()
        # print("Old Label: ", key, "Score: ", score)

        # remap and add to new datapoint if score is above zero
        if score > 0.0:
            non_zero_score = True
            # new labels can be more than one (see aff_remaps) so loop through them and add to the datapoint
            new_keys_list = aff_remaps[sem_class][key]
            for new_key in new_keys_list:
                # if new key already exists in the new datapoint, add the labels to the existing labels (OR operation)
                if new_key in new_datapoint['labels']:
                    new_datapoint['labels'][new_key] = np.logical_or(new_datapoint['labels'][new_key], labels)
                else:
                    new_datapoint['labels'][new_key] = labels
                # print("New Label: ", new_key)

    # Add the new datapoint to the new dataset if there is at least one non-zero score
    if non_zero_score == True:
        new_aff_dataset['data'][sem_class][shape_id] = new_datapoint
        new_aff_dataset['num_objects'] += 1
    else:
        print("No non-zero scores for this datapoint")

No non-zero scores for this datapoint


In [47]:
print("Number of shapes in new training dataset: ", new_aff_dataset['num_objects'])

Number of shapes in new training dataset:  152


In [48]:
# Optional: Save the new dataset
import pickle as pkl
with open(opj('data_root', 'remapped_full_shape_%s_data.pkl' % 'train'), 'wb') as f:
    pkl.dump(new_aff_dataset, f)